# 1개의 열만 고정하고 나머지 열을 행으로 바꾸기

In [ ]:
import pandas as pd
pew = pd.read_csv('../data/pew.csv')
print(pew)

In [ ]:
print(pew.iloc[:, 0:6])

#                    religion  <$10k  $10-20k  $20-30k  $30-40k  $40-50k
# 0                  Agnostic     27       34       60       81       76
# 1                   Atheist     12       27       37       52       35

In [ ]:
pew_long = pd.melt(pew, id_vars='religion')
print(pew_long)

#                     religion            variable  value
# 0                   Agnostic               <$10k     27
# ...r5
# 17              Unaffiliated               <$10k    217
# 18                  Agnostic             $10-20k     34
# 19                   Atheist             $10-20k     27
# ...

In [ ]:
# 위에서 variable, value열의 이름을 바꾸는 방법
pew_long = pd.melt(pew, id_vars='religion', var_name='income', value_name='count')
print(pew_long.head())

#              religion income  count
# 0            Agnostic  <$10k     27
# 1             Atheist  <$10k     12
# 2            Buddhist  <$10k     27
# 3            Catholic  <$10k    418
# 4  Don’t know/refused  <$10k     15

# 2개 이상의 열을 고정하고 나머지 열을 행으로 바꾸기

In [ ]:
billboard = pd.read_csv('../data/billboard.csv')
print(billboard.iloc[0:5, 0:16])

In [ ]:
billboard_long = pd.melt(billboard, id_vars=['year', 'artist', 'track', 'time', 'date.entered'], 
                         var_name='week', value_name='rating')
print(billboard_long.head())

# ebola 데이터 집합 살펴보기

In [ ]:
ebola = pd.read_csv('../data/country_timeseries.csv')
print(ebola.columns)

In [ ]:
print(ebola.iloc[:5, [1, 2, 3, 4, 10, 11]])

#    Day  Cases_Guinea  Cases_Liberia  Cases_SierraLeone  Deaths_Guinea  \
# 0  289        2776.0            NaN            10030.0         1786.0  

In [ ]:
ebola_long = pd.melt(ebola, id_vars=['Date', 'Day'])
print(ebola_long.head())

#          Date  Day      variable   value
# 0    1/5/2015  289  Cases_Guinea  2776.0
# 1    1/4/2015  288  Cases_Guinea  2775.0
# 2    1/3/2015  287  Cases_Guinea  2769.0
# 3    1/2/2015  286  Cases_Guinea     NaN
# 4  12/31/2014  284  Cases_Guinea  2730.0

# 열 이름 나누고 데이터 프레임에 추가하기

In [ ]:
variable_split = ebola_long.variable.str.split('_')
print(variable_split[:5])

# 0    [Cases, Guinea]
# 1    [Cases, Guinea]

In [ ]:
print(type(variable_split))  # Series
print(type(variable_split[0]))  # list

In [ ]:
status_values = variable_split.str.get(0)  # 상태
country_values = variable_split.str.get(1)  # 나라
print(status_values[:5])  # 0    Cases
print(country_values[:5])  # 0    Guinea

In [ ]:
# 분리한 문자열을 status, country라는 열 이름으로 데이터프레임에 추가
ebola_long['status'] = status_values
ebola_long['country'] = country_values
print(ebola_long.head())

#          Date  Day      variable   value status country
# 0    1/5/2015  289  Cases_Guinea  2776.0  Cases  Guinea

# concat 메서드를 응용하여 데이터프레임에 열 추가하기

In [ ]:
variable_split = ebola_long.variable.str.split('_', expand=True)  # expand: 여러 열을 한번에 다루기
variable_split.columns = ['status', 'country']
ebola_parsed = pd.concat([ebola_long, variable_split], axis=1)  # axis=1: 가로로 이어붙임
print(ebola_parsed.head())

#          Date  Day      variable   value status country status country
# 0    1/5/2015  289  Cases_Guinea  2776.0  Cases  Guinea  Cases  Guinea

# 기상 데이터의 여러 열을 하나로 정리하기

In [ ]:
weather = pd.read_csv('../data/weather.csv')
print(weather.iloc[:5, :])

#         id  year  month element  d1    d2    d3  d4    d5  d6  ...  d22   d23  \
# 0  MX17004  2010      1    tmax NaN   NaN   NaN NaN   NaN NaN  ...  NaN   NaN  

In [ ]:
# day열에 날짜 열 정리, temp열에 날짜 열의 데이터 정리
# 최고 최저 온도 눈에 잘 띄지 않음
weather_melt = pd.melt(weather, id_vars=['id', 'year', 'month', 'element'], var_name='day', value_name='temp')
print(weather_melt.head())

#         id  year  month element day  temp
# 0  MX17004  2010      1    tmax  d1   NaN

In [ ]:
# pivot_table메서드: 행과 열의 위치 다시 바꿈
# index: 위치를 유지할 열이름 지정, columns: 피벗할 열 이름 지정, values: 새로운 열의 데이터가 될 열 이름 지정
weather_tidy = weather_melt.pivot_table(
index=['id', 'year', 'month', 'element'],
columns='element',
values='temp',
dropna=False
)
print(weather_tidy)

# element                          tmax       tmin
# id      year month element                      
# MX17004 2010 1     tmax     27.800000        NaN
#                    tmin           NaN  14.500000
#              2     tmax     27.750000        NaN
#                    tmin           NaN  13.225000

In [ ]:
# 데이터프레임의 인덱스를 reset_index 메서드로 새로지정
# weather_tidy의 인덱스를 새로 지정함.
weather_tidy_flat = weather_tidy.reset_index()
print(weather_tidy_flat.head())

# element       id  year  month element       tmax    tmin
# 0        MX17004  2010      1    tmax  27.800000     NaN
# 1        MX17004  2010      1    tmin        NaN  14.500
# 2        MX17004  2010      2    tmax  27.750000     NaN

# 빌보드 차트의 중복 데이터 처리하기

In [ ]:
billboard = pd.read_csv('../data/billboard.csv')
billboard_long = pd.melt(billboard, id_vars=['year', 'artist', 'track', 'time', 'date.entered'], var_name='week', value_name='rating')

print(billboard_long.shape)  # (24092, 7)
print(billboard_long.head())
#    year        artist                    track  time date.entered week  rating
# 0  2000         2 Pac  Baby Don't Cry (Keep...  4:22   2000-02-26  wk1    87.0
# 1  2000       2Ge+her  The Hardest Part Of ...  3:15   2000-09-02  wk1    91.0

In [ ]:
print(billboard_long[billboard_long.track == 'Loser'].head())
#       year        artist  track  time date.entered week  rating
# 3     2000  3 Doors Down  Loser  4:24   2000-10-21  wk1    76.0
# 320   2000  3 Doors Down  Loser  4:24   2000-10-21  wk2    76.0

In [ ]:
# 중복되는 값을 가지고 있는 열을 따로 모아 새로운 데이터프레임에 저장
billboard_songs = billboard_long[['year', 'artist', 'track', 'time']]
print(billboard_songs.shape)  # billboard_songs

In [ ]:
# 중복 데이터 제거

In [ ]:
billboard_songs = billboard_songs.drop_duplicates()
print(billboard_songs.shape)  # (317, 4)

In [ ]:
# 중복을 제거한 데ㅣ터프레임에 id추가
billboard_songs['id'] = range(len(billboard_songs))
print(billboard_songs.head(n=10))

#    year          artist                    track  time  id
# 0  2000           2 Pac  Baby Don't Cry (Keep...  4:22   0
# 1  2000         2Ge+her  The Hardest Part Of ...  3:15   1
# 2  2000    3 Doors Down               Kryptonite  3:53   2
# 3  2000    3 Doors Down                    Loser  4:24   3

In [ ]:
# merge메서드를 사용해 노래 정보와 주간순위데이터 합치기
billboard_ratings = billboard_long.merge(billboard_songs, on=['year', 'artist', 'track', 'time'])
        # billboard_long: 노래 정보
print(billboard_ratings.shape)  # (24092, 8)

# 뉴욕 택시 데이터 준비

In [ ]:
# 데이터 내려받기
import os
import urllib.request

with open('../data/raw_data_urls.txt', 'r') as data_urls:
    for line, url in enumerate(data_urls):
        if line == 5:
            break
        fn = url.split('/')[-1].strip()
        fp = os.path.join('', '../data', fn)
        print(url)
        print(fp)
        urllib.request.urlretrieve(url, fp)

In [ ]:
import glob
nyc_taxi_data = glob.glob('../data/fhv_*')
print(nyc_taxi_data)

In [ ]:
taxi1 = pd.read_csv(nyc_taxi_data[0])
taxi2 = pd.read_csv(nyc_taxi_data[1])
taxi3 = pd.read_csv(nyc_taxi_data[2])
taxi4 = pd.read_csv(nyc_taxi_data[3])
taxi5 = pd.read_csv(nyc_taxi_data[4])

In [ ]:
print(taxi1.head(n=2))
print(taxi2.head(n=2))
print(taxi3.head(n=2))
print(taxi4.head(n=2))
print(taxi5.head(n=2))

In [ ]:
# 데이터프레임 연결
taxi = pd.concat([taxi1, taxi2, taxi3, taxi4, taxi5])
print(taxi.shape)  # (17367717, 3)

# 반복문으로 데이터 준비하기

In [ ]:
list_taxi_df = []

for csv_filename in nyc_taxi_data:
    df = pd.read_csv(csv_filename)
    list_taxi_df.append(df)

print(len(list_taxi_df))  # 5

print(type(list_taxi_df[0]))  # dataframe

taxi_loop_concat = pd.concat(list_taxi_df)
print(taxi_loop_concat.shape)  # (17367717, 3)

print(taxi.equals(taxi_loop_concat))  # Ture
    # 앞의 taxi에 저장된 데이터의 taxi_loop_concat에 저장한 데이터가 같은지 확인
